In [2]:
# Importing Libraries
from langchain_xai import ChatXAI
#from langchain_openai import OpenAI

In [ ]:
# Initialize the model
llm = ChatXAI(model='grok-2-latest', 
              temperature=0.01, 
              max_tokens=None, 
              timeout=None, 
              max_retries=2, 
              api_key='your api key')

#llm = OpenAI(model='gpt-4o') 

In [29]:
# Generate answers to a question
question = "When did India get Independence?"
response = llm.predict(question)
print(response)

India gained independence from British rule on August 15, 1947.


In [31]:
# Generate answers to a question
question = "Who is Sachin Tendulkar"
response = llm.predict(question)
print(response)

Sachin Tendulkar is a former Indian cricketer widely regarded as one of the greatest cricketers of all time. Born on April 24, 1973, in Mumbai, India, he made his debut for the Indian cricket team at the age of 16 in 1989. Throughout his illustrious career, Tendulkar played 200 Test matches and 463 One Day Internationals (ODIs), scoring 15,921 runs in Tests and 18,426 runs in ODIs, both of which are world records.

Known as the "Master Blaster," Tendulkar was renowned for his technique, consistency, and ability to play under pressure. He holds numerous records, including the most runs in international cricket, the most centuries in both Test and ODI cricket, and being the first player to score a double century in an ODI.

Tendulkar retired from cricket in 2013, leaving behind a legacy that continues to inspire millions. He was awarded the Bharat Ratna, India's highest civilian honor, in 2014, becoming the first sportsperson to receive this award. Post-retirement, he has been involved i

In [33]:
# Generate answers to a question
question = "The movie was very bad, but Vicky Kaushals acting was really good, can you classify it as positive/negative/neutral sentiment"
response = llm.predict(question)
print(response)

The sentiment expressed in your statement can be classified as **mixed** or **neutral** overall. Here's the breakdown:

- **Negative sentiment**: "The movie was very bad"
- **Positive sentiment**: "Vicky Kaushal's acting was really good"

The mention of the movie being "very bad" indicates a negative sentiment towards the film as a whole. However, the positive comment about Vicky Kaushal's acting introduces a positive sentiment specifically towards his performance.

When sentiments within a single statement are contradictory, it's often classified as neutral or mixed, as it contains both positive and negative elements.


In [35]:
# Generate answers to a question
question = "What is How are you in Spanish"
response = llm.predict(question)
print(response)

"How are you" in Spanish is "¿Cómo estás?" if you are addressing one person informally, or "¿Cómo está usted?" if you are addressing one person formally. If you are addressing a group, you would say "¿Cómo están?"


## Implementing RAG for custom data (Research Paper Chatbot)

In [37]:
import openai

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
pdf_reader = PyPDFLoader("RAGPaper.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,)
chunks = text_splitter.split_documents(documents)

In [41]:
# Create embeddings using a free HF model
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

C:\Users\User\AppData\Local\Temp\ipykernel_9192\2923984562.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
# Initialize the model
llm = ChatXAI(model='grok-2-latest', 
              temperature=0.01, 
              max_tokens=None, 
              timeout=None, 
              max_retries=2, 
              api_key='your api key')

In [45]:
# Generate answers to a question
question = "When did India get Independence?"
response = llm.predict(question)
print(response)

India gained independence from British rule on August 15, 1947.


In [47]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow question, rephrase the follow up question to be a standalone question.
                                                        Chat History:{chat_history}
                                                        Follow up Input: {question}
                                                        Standalone question:""")
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), condense_question_prompt=CONDENSE_QUESTION_PROMPT, return_source_documents=True,
                                           verbose=False)
                                                        

In [49]:
chat_history=[]
query="""what is a RAG-sequence model?"""
result = qa({"question":query, "chat_history":chat_history})

In [50]:
print(result['answer'])

A RAG-Sequence model is a type of Retrieval-Augmented Generation (RAG) model where the entire output sequence is generated based on a single retrieved document. In this model, the top K documents are retrieved using a retriever, and then the generator produces the output sequence probability for each document. These probabilities are then marginalized to approximate the sequence probability \( p(y|x) \). The model treats the retrieved document as a single latent variable and uses a top-K approximation to compute the sequence probability. 

For decoding, RAG-Sequence does not break into a conventional per-token likelihood, so it uses a method called "Thorough Decoding." This involves running beam search for each document, scoring hypotheses, and then estimating the probability of each hypothesis by running additional forward passes for documents where the hypothesis did not appear in the beam. This process can become computationally intensive for longer output sequences.


In [55]:
chat_history=[]
query="""Can you explain the training setup details explained in this paper?"""
result = qa({"question":query, "chat_history":chat_history})

print(result['answer'])

The training setup details for the models described in the paper are as follows:

- **Training Framework**: All RAG models and BART baselines were trained using Fairseq, a sequence modeling toolkit.

- **Hardware and Precision**: Training was conducted with mixed precision floating-point arithmetic and distributed across 8 NVIDIA V100 GPUs, each with 32GB of memory.

- **Optimization**: The training process involved minimizing the negative marginal log-likelihood of each target using stochastic gradient descent with the Adam optimizer.

- **Document Encoder**: The document encoder, referred to as BERTd, was not updated during training. This decision was made to avoid the need for periodic updates to the document index, which is a costly process. Instead, the document encoder and its index were kept fixed.

- **Fine-Tuning**: Only the query encoder (BERTq) and the BART generator were fine-tuned during the training process.

These details outline the approach taken to train the models ef

In [57]:
chat_history=[]
query="""What are the top 5 references used in this paper?"""
result = qa({"question":query, "chat_history":chat_history})

print(result['answer'])

The top 5 references used in the provided context are:

1. **Stephen Robertson and Hugo Zaragoza. The probabilistic relevance framework: Bm25 and beyond. Found. Trends Inf. Retr., 3(4):333–389, April 2009.**
   - This reference discusses the probabilistic relevance framework, including the BM25 model and its extensions.

2. **Irene Solaiman, Miles Brundage, Jack Clark, Amanda Askell, Ariel Herbert-Voss, Jeff Wu, Alec Radford, and Jian-Bing Wang. Release strategies and the social impacts of language models. ArXiv, abs/1908.09203, 2019.**
   - This reference explores the release strategies and social impacts associated with language models.

3. **Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, and Rob Fergus. End-to-end memory networks. In Advances in Neural Information Processing Systems 28, pages 2440–2448, 2015.**
   - This reference introduces end-to-end memory networks, a type of neural network architecture.

4. **Danqi Chen, Adam Fisch, Jason Weston, and Antoine Bordes. Reading W

## AI Tutor

In [60]:
import openai

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [62]:
pdf_reader = PyPDFLoader("Power BI Ebook.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,)
chunks = text_splitter.split_documents(documents)

In [64]:
# Create embeddings using a free HF model
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

C:\Users\User\AppData\Local\Temp\ipykernel_9192\2923984562.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
# Initialize the model
llm = ChatXAI(model='grok-2-latest', 
              temperature=0.01, 
              max_tokens=None, 
              timeout=None, 
              max_retries=2, 
              api_key='your api key')

In [68]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Behave as a AI tutor, and respond the queries from the document trained, Given the following conversation and a follow question, rephrase the follow up question to be a standalone question.
                                                        Chat History:{chat_history}
                                                        Follow up Input: {question}
                                                        Standalone question:""")
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), condense_question_prompt=CONDENSE_QUESTION_PROMPT, return_source_documents=True,
                                           verbose=False)
                                                        

In [72]:
chat_history=[]
query="""What are the steps to implement RLS?"""
result = qa({"question":query, "chat_history":chat_history})

print(result['answer'])

To implement Row-Level Security (RLS) in Power BI, follow these steps:

1. **Define your roles**: Define the roles for your users, such as sales reps, managers, or regional directors.

2. **Define your rules**: Define the rules that control what data each role can see. For example, you might limit a sales rep to only seeing data for their territory, or a manager to only seeing data for their team.

3. **Implement your rules**: Implement your RLS rules by creating DAX expressions that define the rules for each role. These expressions will be used to filter the data that each user sees.

4. **Test your rules**: Test your RLS rules by previewing the report or dashboard as each role. Ensure that each role only sees the data that they are supposed to see.

Here's an example of how to implement RLS in Power BI:

1. **Create a new role**: Click on the Manage Roles option in the Modeling tab and create a new role, such as "Sales Rep".

2. **Define your rules**: Create a DAX expression that fil